In [16]:
from dataclasses import dataclass, asdict
from typing import List
import re

@dataclass
class Arg:
    type: str
    name: str

    def __repr__(self) -> str:
        return repr((self.type, self.name))
    
@dataclass
class Function:
    name: str
    args: List[Arg]
    ret_type: str
    docs: str
    group: str

    def signature(self) -> str:
        return f'{self.ret_type} {self.name}({", ".join([f"{arg.type} {arg.name}" for arg in self.args])})'

    def to_md(self) -> str:
        return f'### `{self.signature()}`\n\n' + self.docs

def parse_function(line: str, docs) -> Function:
    import re
    m = re.match(r'([a-zA-Z0-9_\*]+)\s+(pkpy_\w+)\s*\((.*)\)', line)
    if m is None:
        return None
    name = m.group(2)
    args = []
    for arg in m.group(3).split(','):
        arg = arg.strip()
        if arg == '':
            continue
        parts = arg.split(' ')
        arg_type = ' '.join(parts[:-1])
        arg_name = parts[-1]
        args.append(Arg(arg_type, arg_name))
    ret_type = m.group(1)

    if 'tvm' in name:
        group = 'tvm'
    elif 'vm' in name:
        group = 'vm'
    elif 'repl' in name:
        group = 'repl'
    else:
        group = 'general'

    return Function(name, args, ret_type, group=group, docs=docs)

In [17]:
path = 'E:/balloonfishmollusc/pocketpy-flutter/src/pocketpy.h'

with open(path, 'r') as file:
  lines = file.readlines()

apis = []

i = 0
while i < len(lines)-1:
    if '__EXPORT' != lines[i].strip():
        i += 1
        continue
    i += 1
    docs = []
    while True:
        doc_line = re.match(r'^\s*///\s*(.*)$', lines[i])
        if not doc_line:
            break
        docs.append(doc_line.group(1))
        i += 1

    f = parse_function(lines[i].strip(' {\n'), '\n'.join(docs))
    assert docs, lines[i]
    if f is not None:
        apis.append(f)
    i += 1

apis.sort(key=lambda x: (x.group, x.name))

In [18]:
from collections import defaultdict

files = defaultdict(list)

for api in apis:
    files[api.group].append(api.to_md())

headers = {
    'vm': r'''---
title: VM
icon: code
order: 10
---''',
    'tvm': r'''---
title: ThreadedVM
icon: code
order: 9
---''',
    'repl': r'''---
title: REPL
icon: code
order: 8
---''',
    'general': r'''---
title: General
icon: code
order: 7
---''',
}

import os
import shutil

folder = 'C-API'

shutil.rmtree(folder)
os.makedirs(folder)

for group, md in files.items():
    with open(f'{folder}/{group}.md', 'w') as f:
        content = '\n\n'.join(md)
        f.write(headers[group] + '\n' + content)

with open(f'{folder}/index.md', 'w') as f:
    desc = '''
The C-API is for developers who want to use PocketPy
in a platform that is not supported by the official yet.

By running the following command on Linux/WSL,
you can get a single header file `pocketpy.h`.

```bash
git clone https://github.com/blueloveth/pocketpy.git
cd pocketpy
python amalgamate.py
```
    '''
    f.write(f'---\nlabel: {folder}\nicon: package\norder: 1\n---\n\n' + desc)